In [ ]:
import os
from datetime import datetime as dt
from matplotlib import pyplot as plt
import numpy as np

from FLplot import getFLpathData
from samuraiAnalysis import getVarLims,gribTools,plotFLpath,samImport,samPlt 

getFLpathData = getFLpathData.getFLpathData
getVarLims = getVarLims.getVarLims
gribImport = gribTools.gribImport
gribLevs = gribTools.gribLevs
plotFLpath = plotFLpath.plotFLpath
samImport = samImport.samImport
plotContour = samPlt.plotContour
plotVec = samPlt.plotVec

%matplotlib inline

In [ ]:
### Which case do you want to run? ###
# This is the name of the directory where the SAMURAI
# output is located
# outPrefix = 'P6_D7_fsdabr'
# outPrefix = 'P6_fsd'
# outPrefix = 'P6_fsdabr'
# outPrefix = 'P6_fsdMultV'
outPrefix = 'P6corr_fsd'
# outPrefix = 'S3_fsd'
# outPrefix = 'S3_fsdabr'
# outPrefix = 'S3_fsdMultV'
# outPrefix = 'S3corr_fsd'
# outPrefix = 'S3P6_fsdabr'
# outPrefix = 'S3prl1'
# outPrefix = 'S3prl1_fsdabr'


### Set various file path variables ###
samPrefix = '/Users/danstechman/GoogleDrive/PECAN-Data/samurai/20150706/'
samFile = samPrefix + outPrefix + '/output/samurai_XYZ_analysis.nc'
savePath = samPrefix + outPrefix + '/figs/'
flFile = '/Users/danstechman/GoogleDrive/PECAN-Data/FlightLevelData/Processed/20150706_FltLvl_Processed.nc'
gribFile = '/Users/danstechman/GoogleDrive/PECAN-Data/model/RAP/20150706/rap_130_20150706_0400_000.nc'

### Import SAMURAI data ###
samData = samImport(samFile)


### Which variables do we want to plot, at what levels (levels are in km), ###
###     cartesian or map coordinates ('xy' or 'map'), and do you want      ###
###     to save the figures?                                               ###
pltDBZ  = True
pltU    = True
pltV    = True
pltW    = True
pltVort = True

pltVec  = True

pltRAP  = False

pltGRlocs = True

savePlt = True

crds = 'map'

pltLevs = samData['alt']
# pltLevs = [1,5,6.5]

### Plot flight track on analysis? ###
# flS and flE indicate the start and end date/time for the track.
# For the data index of pddSprlTimes, use (leg/sprl # - 1).
pltFT   = True
pddSprlTimes = np.load('/Users/danstechman/GoogleDrive/School/Research/PECAN/Radar/code/samuraiAnalysis/pecan_PDD-Sprl_times/20150706_pddSprl_times.npy')[()]

# PDD 6
if 'P6' in outPrefix and 'S3P6' not in outPrefix:
    flS = pddSprlTimes['I20P_strtDT'][5]
    flE = pddSprlTimes['I20P_endDT'][5]

# Spiral 3    
elif 'S3' in outPrefix and not any(val in outPrefix for val in ('S3P6','prl')):# 'S3P6' not in outPrefix and 'prl':
    flS = pddSprlTimes['I20S_strtDT'][2]
    flE = pddSprlTimes['I20S_endDT'][2]

# Spiral3/PDD6 combo
elif 'S3P6' in outPrefix:
    flS = pddSprlTimes['I20P_strtDT'][5]
    flE = pddSprlTimes['I20S_endDT'][2]

#Purl 1 from Spiral 3:
elif 'prl1' in outPrefix:
    flS = dt.strptime('20150706-042710','%Y%m%d-%H%M%S')
    flE = dt.strptime('20150706-043152','%Y%m%d-%H%M%S')

else:
    raise ValueError('Unknown FL path start/end datetimes')

    
##########################################
### End of commonly modified variables ###
##########################################

# Ground-based radar names and locations
grTxt = ['D7']#,'FSD','ABR']
latGR = [43.3864]#,43.583,45.451]
lonGR = [-97.7197]#,-96.724,-98.408]



if not pltRAP:
    runId = outPrefix
else:
    runId = outPrefix + '_rapOvr'

dT = samData['time']

if pltFT:
    flData = getFLpathData(flFile,pathStrt=flS,pathEnd=flE,crdsOnly=True)


if not os.path.exists(savePath):
    os.makedirs(savePath)

if dT is not None:
    dtSave = dt.strftime(dT,'%Y%m%d_%H%M')
else:
    dtSave = 'unknownDT'

if savePlt:
    if runId:
        runIdSv = '_' + runId.replace('_','-')
    
    
if pltRAP:
    ### Import GRIB data and match levels ###
    gribData = gribImport(gribFile)
    rapLevIx = gribLevs(pltLevs,gribData['geoHght'])

    
for ix in range(0,len(pltLevs)):
    lev = pltLevs[ix]
    
    print('Now plotting {:.2f} km ({} of {})...'.format(lev,ix+1,len(pltLevs)))

    if pltDBZ:
        fig,ax,grd,proj = plotContour(samData['dbz'],lev,samData['lon'],samData['lat'],
                                      crds,'dbz',figsize=(10,10),dT=dT,vLimMthd='default',
                                      runId=runId)
        
        if pltFT and crds is 'map':
            plotFLpath(proj,flData['lon'],flData['lat'],dubLine=True)
            
        if pltVec:
            plotVec(samData['lon'][1::5],samData['lat'][1::5],samData['u'][lev*2,1::5,1::5],
                   samData['v'][lev*2,1::5,1::5],crds,proj=proj)
            
        if pltRAP and crds is 'map':
            plotVec(gribData['lon'],gribData['lat'],gribData['u'][rapLevIx[ix],:,:],
                    gribData['v'][rapLevIx[ix],:,:],crds,proj=proj,color='white',
                    linewidth=1.5,edgecolor='white',quivKeyX=0.08)
            plotVec(gribData['lon'],gribData['lat'],gribData['u'][rapLevIx[ix],:,:],
                    gribData['v'][rapLevIx[ix],:,:],crds,proj=proj,color='blue',
                    edgecolor=None,quivKeyX=0.08)
            
        if pltGRlocs and crds is 'map' and 'D7' in runId:
            plt.scatter(lonGR,latGR,marker='d',color='w',s=200,zorder=7,transform=proj)
            plt.scatter(lonGR,latGR,marker='d',color='k',s=100,zorder=8,transform=proj)

            for labelGR, c, d in zip(grTxt, lonGR, latGR):
                plt.annotate(labelGR, xy = (c, d), xytext = (-8, 5),zorder=10,
                             textcoords = 'offset points',fontsize=13,color='w',
                             bbox=dict(boxstyle="round", fc="b",alpha=0.6,pad=0.01),
                             transform=proj)
        
        if savePlt:
            saveStr = '{}{}{}_{}_{}_{:.1f}km.png'.format(savePath,dtSave,runIdSv,crds,'dbz',lev)
            fig.savefig(saveStr,bbox_inches='tight')
            plt.close()
         
        
        

    if pltU:
        fig,ax,grd,proj = plotContour(samData['u'],lev,samData['lon'],samData['lat'],
                                      crds,'u',figsize=(10,10),dT=dT,vLimMthd='default',
                                      runId=runId)
        
        if pltFT and crds is 'map':
            plotFLpath(proj,flData['lon'],flData['lat'],dubLine=True)
            
        if pltVec:
            plotVec(samData['lon'][1::5],samData['lat'][1::5],samData['u'][lev*2,1::5,1::5],
                   samData['v'][lev*2,1::5,1::5],crds,proj=proj)
            
        if pltRAP and crds is 'map':
            plotVec(gribData['lon'],gribData['lat'],gribData['u'][rapLevIx[ix],:,:],
                    gribData['v'][rapLevIx[ix],:,:],crds,proj=proj,color='white',
                    linewidth=1.5,edgecolor='white',quivKeyX=0.08)
            plotVec(gribData['lon'],gribData['lat'],gribData['u'][rapLevIx[ix],:,:],
                    gribData['v'][rapLevIx[ix],:,:],crds,proj=proj,color='blue',
                    edgecolor=None,quivKeyX=0.08)
            
        if pltGRlocs and crds is 'map' and 'D7' in runId:
            plt.scatter(lonGR,latGR,marker='d',color='w',s=200,zorder=7,transform=proj)
            plt.scatter(lonGR,latGR,marker='d',color='k',s=100,zorder=8,transform=proj)

            for labelGR, c, d in zip(grTxt, lonGR, latGR):
                plt.annotate(labelGR, xy = (c, d), xytext = (-8, 5),zorder=10,
                             textcoords = 'offset points',fontsize=13,color='w',
                             bbox=dict(boxstyle="round", fc="b",alpha=0.6,pad=0.01),
                             transform=proj)
        
        if savePlt:
            saveStr = '{}{}{}_{}_{}_{:.1f}km.png'.format(savePath,dtSave,runIdSv,crds,'u',lev)
            fig.savefig(saveStr,bbox_inches='tight')
            plt.close()
    
    
    

    if pltV:
        fig,ax,grd,proj = plotContour(samData['v'],lev,samData['lon'],samData['lat'],
                                      crds,'v',figsize=(10,10),dT=dT,vLimMthd='default',
                                      runId=runId)
        
        if pltFT and crds is 'map':
            plotFLpath(proj,flData['lon'],flData['lat'],dubLine=True)
            
        if pltVec:
            plotVec(samData['lon'][1::5],samData['lat'][1::5],samData['u'][lev*2,1::5,1::5],
                   samData['v'][lev*2,1::5,1::5],crds,proj=proj)
            
        if pltRAP and crds is 'map':
            plotVec(gribData['lon'],gribData['lat'],gribData['u'][rapLevIx[ix],:,:],
                    gribData['v'][rapLevIx[ix],:,:],crds,proj=proj,color='white',
                    linewidth=1.5,edgecolor='white',quivKeyX=0.08)
            plotVec(gribData['lon'],gribData['lat'],gribData['u'][rapLevIx[ix],:,:],
                    gribData['v'][rapLevIx[ix],:,:],crds,proj=proj,color='blue',
                    edgecolor=None,quivKeyX=0.08)
            
        if pltGRlocs and crds is 'map' and 'D7' in runId:
            plt.scatter(lonGR,latGR,marker='d',color='w',s=200,zorder=7,transform=proj)
            plt.scatter(lonGR,latGR,marker='d',color='k',s=100,zorder=8,transform=proj)

            for labelGR, c, d in zip(grTxt, lonGR, latGR):
                plt.annotate(labelGR, xy = (c, d), xytext = (-8, 5),zorder=10,
                             textcoords = 'offset points',fontsize=13,color='w',
                             bbox=dict(boxstyle="round", fc="b",alpha=0.6,pad=0.01),
                             transform=proj)
        
        if savePlt:
            saveStr = '{}{}{}_{}_{}_{:.1f}km.png'.format(savePath,dtSave,runIdSv,crds,'v',lev)
            fig.savefig(saveStr,bbox_inches='tight')
            plt.close()
    
    
    

    if pltW:
        fig,ax,grd,proj = plotContour(samData['w'],lev,samData['lon'],samData['lat'],
                                      crds,'w',figsize=(10,10),dT=dT,vLimMthd='default',
                                      runId=runId)
        
        if pltFT and crds is 'map':
            plotFLpath(proj,flData['lon'],flData['lat'],dubLine=True)
            
        if pltVec:
            plotVec(samData['lon'][1::5],samData['lat'][1::5],samData['u'][lev*2,1::5,1::5],
                   samData['v'][lev*2,1::5,1::5],crds,proj=proj)
            
        if pltRAP and crds is 'map':
            plotVec(gribData['lon'],gribData['lat'],gribData['u'][rapLevIx[ix],:,:],
                    gribData['v'][rapLevIx[ix],:,:],crds,proj=proj,color='white',
                    linewidth=1.5,edgecolor='white',quivKeyX=0.08)
            plotVec(gribData['lon'],gribData['lat'],gribData['u'][rapLevIx[ix],:,:],
                    gribData['v'][rapLevIx[ix],:,:],crds,proj=proj,color='blue',
                    edgecolor=None,quivKeyX=0.08)
            
        if pltGRlocs and crds is 'map' and 'D7' in runId:
            plt.scatter(lonGR,latGR,marker='d',color='w',s=200,zorder=7,transform=proj)
            plt.scatter(lonGR,latGR,marker='d',color='k',s=100,zorder=8,transform=proj)

            for labelGR, c, d in zip(grTxt, lonGR, latGR):
                plt.annotate(labelGR, xy = (c, d), xytext = (-8, 5),zorder=10,
                             textcoords = 'offset points',fontsize=13,color='w',
                             bbox=dict(boxstyle="round", fc="b",alpha=0.6,pad=0.01),
                             transform=proj)
        
        if savePlt:
            saveStr = '{}{}{}_{}_{}_{:.1f}km.png'.format(savePath,dtSave,runIdSv,crds,'w',lev)
            fig.savefig(saveStr,bbox_inches='tight')
            plt.close()
    
    
    

    if pltVort:
        fig,ax,grd,proj = plotContour(samData['vort'],lev,samData['lon'],samData['lat'],
                                      crds,'vort',figsize=(10,10),dT=dT,vLimMthd='default',
                                      runId=runId)
        
        if pltFT and crds is 'map':
            plotFLpath(proj,flData['lon'],flData['lat'],dubLine=True)
            
        if pltVec:
            plotVec(samData['lon'][1::5],samData['lat'][1::5],samData['u'][lev*2,1::5,1::5],
                   samData['v'][lev*2,1::5,1::5],crds,proj=proj)
            
        if pltRAP and crds is 'map':
            plotVec(gribData['lon'],gribData['lat'],gribData['u'][rapLevIx[ix],:,:],
                    gribData['v'][rapLevIx[ix],:,:],crds,proj=proj,color='white',
                    linewidth=1.5,edgecolor='white',quivKeyX=0.08)
            plotVec(gribData['lon'],gribData['lat'],gribData['u'][rapLevIx[ix],:,:],
                    gribData['v'][rapLevIx[ix],:,:],crds,proj=proj,color='blue',
                    edgecolor=None,quivKeyX=0.08)
            
        if pltGRlocs and crds is 'map' and 'D7' in runId:
            plt.scatter(lonGR,latGR,marker='d',color='w',s=200,zorder=7,transform=proj)
            plt.scatter(lonGR,latGR,marker='d',color='k',s=100,zorder=8,transform=proj)

            for labelGR, c, d in zip(grTxt, lonGR, latGR):
                plt.annotate(labelGR, xy = (c, d), xytext = (-8, 5),zorder=10,
                             textcoords = 'offset points',fontsize=13,color='w',
                             bbox=dict(boxstyle="round", fc="b",alpha=0.6,pad=0.01),
                             transform=proj)
        
        if savePlt:
            saveStr = '{}{}{}_{}_{}_{:.1f}km.png'.format(savePath,dtSave,runIdSv,crds,'vort',lev)
            fig.savefig(saveStr,bbox_inches='tight')
            plt.close()